# Coletor de dados dos *Running Shoes* do site RunRepeat

---
### Rodrigo Fragoso 
- [**Linkedin**](https://www.linkedin.com/in/rodrigo-a-fragoso/) <br/>
- **Email** : rodrigoandradefragoso@gmail.com <br/>

### Resumo
-  Após extrair os links referentes a cada tênis, nós iremos acessar a sua página e colher todas as informações disponíveis sobre eles ;
-  Os outputs esperados são diversas informações sobre cada tênis: desde preços até *reviews* feitos pela comunidade.
---

<a id='top'></a>
## Sumário

[1 - Importações das bibliotecas](#t1)

[2 - Coleta dos dados da página de busca](#t2)

[3 - Processamento dos dados brutos](#t3)

[4 - Verificação do resultado](#t4)


##     

<a id='t1'></a>
## 1 - Importações das bibliotecas
- [Sumário](#top)   
    - [Próximo](#t2)

### Para iniciarmos a extração será necessário o uso de algumas bibliotecas específicas, que serão importadas na célula abaixo:
-  Pandas: ferramenta rápida e poderosa, responsável pela manipulação/analise de dados através do formato *dataframe* ;
-  re: modulo para realizar operações de correspondência (em texto) através de expressões regulares ;
-  time: modulo utilizado, principalmente, para cálculo de tempo de processamento e criação de *delays* ;
-  requests:  biblioteca HTTP utilizada para fazer o download do código fonte da página ;
-  bs4 (Beautiful Soup 4): biblioteca utilizada para extrair dados de arquivos HTML e XML, utilizada como *parser* para navegarmos dentro dos arquivos criados pela requests ;
-  tqdm: utilizada para acompanhar a duração de loops ;
-  glob: modulo responsável por identificar arquivos em uma pasta ;

In [1]:
import pandas as pd
import re
import time

import requests as rq
import bs4 as bs4
import tqdm
import glob
import json

##     

<a id='t2'></a>
## 2 - Coleta de dados da página do tênis
- [Sumário](#top) 
    - [Anterior](#t1)
    - [Próximo](#t3)

### Nesta etapa, iremos navegar pela página de cada tênis e extrair o maximo de informações possiveis que possam descrevê-lo de alguma maneira analítica.
-  Com o arquivo criado no primeiro processo, resgataremos o link e o nome de cada tênis ;
-  O cabeçalho da *dataframe* será observado para garantir que ele foi carregado corretamente.

In [2]:
df = pd.read_json("./dados_json/parsed_running_shoes.json", lines=True)
df.head(7)

,link,name
0,/brooks-adrenaline-gts-19?selected_color=422261,Brooks Adrenaline GTS 19
1,/brooks-adrenaline-gts-19?selected_color=422261,Brooks Adrenaline GTS 19
2,/brooks-ghost-12?selected_color=799577,Brooks Ghost 12
3,/brooks-ghost-12?selected_color=799577,Brooks Ghost 12
4,/brooks-glycerin-17?selected_color=642780,Brooks Glycerin 17
5,/brooks-glycerin-17?selected_color=642780,Brooks Glycerin 17
6,/nike-air-zoom-pegasus-36?selected_color=712129,Nike Air Zoom Pegasus 36


### Como podemos perceber, existem links e nomes duplicados. Para resolver este problema utilizaremos o método drop_duplicates() do pandas, no qual eliminamos os registros repetidos.
-  O sort_values() é utilizado para ordenar de acordo com os nomes dos calçados para um futuro merge com os dados extraídos, assim como o reset_index() que irá resetar o index e deixá-lo em ordem crescente ;
-  Feito isso, encontramos uma lista de 2277 tênis para serem explorados.


In [3]:
lista_de_links = df.drop_duplicates().sort_values(by='name').reset_index()
len(lista_de_links)

2277

In [4]:
lista_de_links.head()

,index,link,name
0,2196,/361-degrees-chaser-2?selected_color=334581,361 Degrees Chaser 2
1,4366,/361-degrees-enjector?selected_color=388471,361 Degrees Enjector
2,4152,/361-degrees-feisu?selected_color=373771,361 Degrees Feisu
3,2622,/361-degrees-kroozer?selected_color=388609,361 Degrees Kroozer
4,1460,/361-degrees-meraki?selected_color=253904,361 Degrees Meraki


### Com essa lista, podemos baixar as páginas desejadas.
-  Cada tênis teve sua página salva em um arquivo ;
-  Foi utilizado o método iterrows() para iterar entre todos os valores do link e do nome do produto no dataframe ;
-  Ao salvar o arquivo, foi utilizado o regex para tratar o nome do link e torná-lo compatível com os caracteres permitidos pelo windows.

In [5]:
url = "https://runrepeat.com{link}"

for index, row in lista_de_links.iterrows():
    print(row['name'],'   ', row['link'])
    
    urll = url.format(link=row['link'])

    response = rq.get(urll)
    
    link_name_pre = row['name'].replace(' ','_')
    link_name = link_name_pre.replace('/','_')
    

    with open("./dados_brutos/shoes_{}.html".format(link_name), 'w+',encoding="utf-8") as output:
        output.write(response.text)
    time.sleep(0.3)

361 Degrees Chaser 2     /361-degrees-chaser-2?selected_color=334581
361 Degrees Enjector     /361-degrees-enjector?selected_color=388471
361 Degrees Feisu     /361-degrees-feisu?selected_color=373771
361 Degrees Kroozer     /361-degrees-kroozer?selected_color=388609
361 Degrees Meraki     /361-degrees-meraki?selected_color=253904
361 Degrees Meraki 2     /361-degrees-meraki-2?selected_color=691383
361 Degrees Nemesis     /361-degrees-nemesis
361 Degrees Ortega 2     /361-degrees-ortega-2?selected_color=339076
361 Degrees Pacer ST     /361-degrees-pacer-st
361 Degrees Santiago     /361-degrees-santiago?selected_color=254077
361 Degrees Sensation 3     /361-degrees-sensation-3?selected_color=338885
361 Degrees Sensation 4     /361-degrees-sensation-4?selected_color=840788
361 Degrees Spinject     /361-degrees-spinject?selected_color=338863
361 Degrees Spire 3     /361-degrees-spire-3?selected_color=388447
361 Degrees Spire 4     /361-degrees-spire-4?selected_color=885890
361 Degrees Str

##     

<a id='t3'></a>
## 3 - Processamento dos dados brutos
- [Sumário](#top) 
    - [Anterior](#t2)
    - [Próximo](#t4)

### Ao navegar pelas páginas, podemos perceber que existe uma grande diversidade de informações, como esperado do RunRepeat. Dito isto, foram feitas diversas tentativas e testes para chegar nas *tags* desejadas. 
-  No glob, é utilizada a função sorted() para ordenar os arquivos da mesma maneira que foi ordenada a lista de links ;
-  O código pode parecer complexo mas se trata apenas de uma lógica para extrair os dados necessários através do código html, a depender da página este processo pode ser muito rapido ou demorar horas/dias ;
-  O regex também foi utilizado para auxiliar na procura de textos específicos ;
-  Por fim, os dados brutos foram salvos em JSON.


In [6]:
with open("./dados_json/parsed_shoes_info.json", 'w+') as output:
    for shoes_file in tqdm.tqdm_notebook(sorted(glob.glob("./dados_brutos/shoes*"))):
        with open(shoes_file, 'r+',encoding="utf-8") as inp:
            page_html = inp.read()
            parsed = bs4.BeautifulSoup(page_html, 'html.parser')
            
            class_image= parsed.find_all('img')
            class_value = parsed.find_all(attrs={"class":re.compile(r"-value")})
            class_fact = parsed.find_all(attrs={"class":re.compile(r"-fact")})
            class_ranktext= parsed.find_all(attrs={"class":re.compile(r"rank-text")})
            class_expertreview = parsed.find_all(attrs={"class":re.compile(r"rr-reviews-score-average")})
            class_reasons=parsed.find_all(attrs={"class":re.compile(r"gb-w-title")})
            class_good=parsed.find_all(attrs={"id":"the_good"})
            class_bad=parsed.find_all(attrs={"id":"the_bad"})

            data = dict()
            
            for e in class_image:
                if e.has_attr('src'):
                    colname = 'image'
                    data[colname] = e['src']
                break
            
            for e in class_value:
                colname = "_".join(e['class'])
                data[colname] = e.text.strip()

            for e in class_fact:
                if e.text.strip() != None:
                    colname =  e.text.strip()
                    if e.find("span",{"class":re.compile(r"rating-fact-bar-value-(\d+)")}) != None:
                        data[colname] = e.find("span",{"class":re.compile("rating-fact-bar-value-*")})['class'][1]
                        
                        
            for e in class_fact:
                if e.find("span",{"class":"label-rating-fact"}) != None:
                    colname =  e.find("span",{"class":"label-rating-fact"}).text.strip()
                    if e.find("span",{"class":"rating-value"}) != None:
                        data[colname] = e.find("span",{"class":"rating-value"}).text.strip()
                        
            for e in class_ranktext:
                colname = e.text.replace('\n','').replace('           ',' ').strip()
                data[colname] = 1

            for e in class_expertreview:
                colname = "_".join(e['class'])
                data[colname] = e.text.strip()
            
            for e in class_reasons:
                if re.compile(r'(\d+)').match(e.text.replace('\n','')) != None:
                    colname = "_".join(re.compile('[a-z]+').findall(e.text.replace('\n','')))
                    data[colname] = re.compile(r'(\d+)').match(e.text.replace('\n','')).group()
               
            write=""
            for e in class_good:
                if e.find_all("li") != None:
                    texts=e.find_all("li")
                    for textss in texts:
                        write= write + " " + textss.text.strip()         
                    colname = "good_reasons_to_buy"
                    data[colname] = write

            write=""
            for e in class_bad:
                if e.find_all("li") != None:
                    texts=e.find_all("li")
                    for textss in texts:
                        write= write + " " + textss.text.strip()         
                    colname = "bad_reasons_to_buy"
                    data[colname] = write                    

            output.write("{}\n".format(json.dumps(data)))

#### Neste passo, focamos em encontrar *features* que melhor representem o produto, além de trazer analises do próprio site que podem ser muito ricas.
- Feito isto, é necessário adicionar o link do tênis e o seu nome atráves da lista utilizada para acessar as páginas, como os valores estão ordenados pela mesma lógica é possível adicionar a coluna diretamente sem maiores tratamentos.

In [7]:
df = pd.read_json("./dados_json/parsed_shoes_info.json", lines=True)
df['link']= lista_de_links['link']
df['shoes_name']= lista_de_links['name']

In [8]:
df.to_json("./dados_json/parsed_shoes_info.json",orient='records', lines=True)

##     

<a id='t4'></a>
## 4 - Verificação do resultado
- [Sumário](#top)   
    - [Anterior](#t3)

### Para finalizar este processo, é feita uma verificação dos dados obtidos.
-  Para visualização transformamos o arquivo JSON no formato *dataframe* do pandas e verificamos os 5 primeiros e últimos registros ;
-  Os dados foram coletados com êxito ;
-  Percebemos que existem diversas *features* (XXX colunas), com diferentes tipos de dados. O tratamento será feito na próxima etapa.

In [9]:
df = pd.read_json("./dados_json/parsed_shoes_info.json", lines=True)
df.shape

(2277, 744)

In [10]:
df.head()

,,A popular pick,A top 1% best Road running shoe,A top 1% best Trail running shoe,A top 10% best Road running shoe,A top 10% best Trail running shoe,A top 2% best Road running shoe,A top 2% best Trail running shoe,A top 3% best Road running shoe,A top 3% best Trail running shoe,...,special-editions-value,strike-pattern-value,technology-value,terrain-value,type-value,update-value,use-value,waterproofing-value,weight-value,width-value
0,rating-fact-bar-value-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Midfoot strike,None,Road,None,None,None,None,Men: 232g | Women: 193g,Normal | Normal
1,rating-fact-bar-value-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Midfoot strike,None,Road,None,None,Jogging,None,Men: 281g | Women: 230g,Normal | Normal
2,rating-fact-bar-value-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Midfoot strike,None,Road,None,None,None,None,Men: 201g | Women: 162g,Normal | Normal
3,rating-fact-bar-value-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Midfoot strike,None,Road,None,None,Jogging,None,Men: 283g | Women: 283g,"Normal, Wide | Normal, Wide"
4,rating-fact-bar-value-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Midfoot strike,None,Road,None,361 Degrees Meraki 2,Jogging,None,Men: 289g | Women: 241g,"Normal, Wide, X-Wide | Normal, Wide"


In [11]:
df.tail(5)

,,A popular pick,A top 1% best Road running shoe,A top 1% best Trail running shoe,A top 10% best Road running shoe,A top 10% best Trail running shoe,A top 2% best Road running shoe,A top 2% best Trail running shoe,A top 3% best Road running shoe,A top 3% best Trail running shoe,...,special-editions-value,strike-pattern-value,technology-value,terrain-value,type-value,update-value,use-value,waterproofing-value,weight-value,width-value
2272,rating-fact-bar-value-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Midfoot strike,None,Road,None,,None,None,Men: 210g | Women: 210g,Normal | Normal
2273,rating-fact-bar-value-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Heel strike,None,Road,None,,Jogging,None,Men: 281g | Women: 235g,Normal | Normal
2274,rating-fact-bar-value-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Forefoot strike,None,Road,Low drop,None,None,None,Men: 213g | Women: 184g,Normal | Normal
2275,rating-fact-bar-value-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Heel strike,None,Road,None,None,None,None,Men: 240g | Women: 226g,Normal | Normal
2276,rating-fact-bar-value-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,Heel strike,None,Road,None,,Jogging,None,Men: 255g | Women: 240g,Normal | Normal
